In [2]:
from pathlib import Path
!pip install gradio
import gradio as gr
! pip install stable_diffusion_videos[realesrgan]
from stable_diffusion_videos import generate_images

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import torch

from stable_diffusion_videos import StableDiffusionWalkPipeline

pipeline = StableDiffusionWalkPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16,
    revision="fp16",
).to("cuda")

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [5]:
class Interface:
    def __init__(self, pipeline):
        self.pipeline = pipeline
        self.interface_images = gr.Interface(
            self.fn_images,
            inputs=[

                gr.Textbox("SteamPunk airship Bursting through the clouds, cyberpunk art", label='Prompt'),
                gr.Slider(1, 24, 1, step=1, label='Batch size'),
                gr.Slider(1, 16, 1, step=1, label='# Batches'),
                gr.Slider(10, 100, 50, step=1, label='# Inference Steps'),
                gr.Slider(5.0, 15.0, 7.5, step=0.5, label='Guidance Scale'),
                gr.Slider(512, 1024, 512, step=64, label='Height'),
                gr.Slider(512, 1024, 512, step=64, label='Width'),
                gr.Checkbox(False, label='Upsample'),
                gr.Textbox("./images", label='Output directory to save results to'),
                # gr.Checkbox(False, label='Push results to Hugging Face Hub'),
                # gr.Textbox("", label='Hugging Face Repo ID to push images to'),
            ],
            outputs=gr.Gallery(),
        )

        self.interface_videos = gr.Interface(
            self.fn_videos,
            inputs=[

                gr.Textbox("blueberry spaghetti\nstrawberry spaghetti", lines=2, label='Prompts, separated by new line'),
                gr.Textbox("42\n1337", lines=2, label='Seeds, separated by new line'),
                gr.Slider(3, 1000, 5, step=1, label='# Interpolation Steps between prompts'),
                gr.Slider(3, 60, 5, step=1, label='Output Video FPS'),
                gr.Slider(1, 24, 1, step=1, label='Batch size'),
                gr.Slider(10, 100, 50, step=1, label='# Inference Steps'),
                gr.Slider(5.0, 15.0, 7.5, step=0.5, label='Guidance Scale'),
                gr.Slider(512, 1024, 512, step=64, label='Height'),
                gr.Slider(512, 1024, 512, step=64, label='Width'),
                gr.Checkbox(False, label='Upsample'),
                gr.Textbox("./dreams", label='Output directory to save results to'),
            ],
            outputs=gr.Video(),
        )
        self.interface = gr.TabbedInterface(
            [self.interface_images, self.interface_videos],
            ['Images!', 'Videos!'],
        )

    def fn_videos(
        self,
        prompts,
        seeds,
        num_interpolation_steps,
        fps,
        batch_size,
        num_inference_steps,
        guidance_scale,
        height,
        width,
        upsample,
        output_dir,
    ):
        prompts = [x.strip() for x in prompts.split('\n') if x.strip()]
        seeds = [int(x.strip()) for x in seeds.split('\n') if x.strip()]

        return self.pipeline.walk(
            prompts=prompts,
            seeds=seeds,
            num_interpolation_steps=num_interpolation_steps,
            fps=fps,
            height=height,
            width=width,
            output_dir=output_dir,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps,
            upsample=upsample,
            batch_size=batch_size
        )

    def fn_images(
        self,
        prompt,
        batch_size,
        num_batches,
        num_inference_steps,
        guidance_scale,
        height,
        width,
        upsample,
        output_dir,
        repo_id=None,
        push_to_hub=False,
    ):
        image_filepaths = generate_images(
            self.pipeline,
            prompt,
            batch_size=batch_size,
            num_batches=num_batches,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            output_dir=output_dir,
            image_file_ext='.jpg',
            upsample=upsample,
            height=height,
            width=width,
            push_to_hub=push_to_hub,
            repo_id=repo_id,
            create_pr=False,
        )
        return [(x, Path(x).stem) for x in sorted(image_filepaths)]

    def launch(self, *args, **kwargs):
        self.interface.launch(*args, **kwargs)

In [ ]:
interface = Interface(pipeline)
interface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>